# Code Generator

The requirement: use a Frontier model to generate high performance C++ code from Python code


In [1]:
# imports

import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess
import google.generativeai as genai

In [2]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
# os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['GEMINI_API_KEY'] = os.getenv('GEMINI_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [3]:
# initialize
# NOTE - option to use ultra-low cost models by uncommenting last 2 lines

openai = OpenAI()
# claude = anthropic.Anthropic()
# OPENAI_MODEL = "gpt-4o"
# CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

# Want to keep costs ultra-low? Uncomment these lines:
OPENAI_MODEL = "gpt-4o-mini"
# CLAUDE_MODEL = "claude-3-haiku-20240307"

GEMINI_MODEL = "gemini-2.0-flash-exp"
gemini_model_instance = genai.GenerativeModel(GEMINI_MODEL)


In [4]:
system_message = "You are an assistant that reimplements Python code in high performance C++ for Windows. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time."

In [5]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work other than a few comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip or cstdint.\n\n"
    user_prompt += python
    return user_prompt

In [6]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [7]:
# write to a file called optimized.cpp

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("optimized.cpp", "w") as f:
        f.write(code)

In [8]:
def optimize_gpt(python):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        print(fragment, end='', flush=True)
    write_output(reply)

In [9]:
# def optimize_claude(python):
#     result = claude.messages.stream(
#         model=CLAUDE_MODEL,
#         max_tokens=2000,
#         system=system_message,
#         messages=[{"role": "user", "content": user_prompt_for(python)}],
#     )
#     reply = ""
#     with result as stream:
#         for text in stream.text_stream:
#             reply += text
#             print(text, end="", flush=True)
#     write_output(reply)

In [10]:
def optimize_gemini(python):
    if gemini_model_instance is None:
        return

    prompt = user_prompt_for(python)

    try:
        if hasattr(gemini_model_instance, '_system_instruction') and gemini_model_instance._system_instruction:
             stream = gemini_model_instance.generate_content(prompt, stream=True)
        else:
             generation_config = genai.types.GenerationConfig(
                 # max_output_tokens=2048,
                 temperature=0.2
             )
             safety_settings = [
                {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
                {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
             ]

             stream = gemini_model_instance.generate_content(
                 prompt,
                 generation_config=generation_config,
                 safety_settings=safety_settings,
                 stream=True
            )

        reply = ""
        for chunk in stream:
            try:
                fragment = chunk.text
                reply += fragment
                print(fragment, end='', flush=True)
            except ValueError:
                pass
            except Exception as chunk_error:
                print(f"\nError process: {chunk_error}")


        if not reply.endswith('\n'):
            print()

        write_output(reply)

    except genai.types.generation_types.BlockedPromptException as bpe:
         print(f"\nError: Prompt was blocked by Gemini. {bpe}")
    except genai.types.generation_types.StopCandidateException as sce:
         print(f"\nError: Generation stopped by Gemini (e.g. safety). {sce}")
         write_output(reply)
    except Exception as e:
        print(f"\nError when calling Google Gemini: {e}")
        import traceback
        traceback.print_exc()

In [11]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [12]:
exec(pi)

Result: 3.141592658589
Execution Time: 16.262544 seconds


In [13]:
optimize_gpt(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>
#include <cstdint>

double calculate(int64_t iterations, double param1, double param2) {
    double result = 1.0;
    for (int64_t i = 1; i <= iterations; ++i) {
        double j = i * param1 - param2;
        result -= (1.0 / j);
        j = i * param1 + param2;
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = std::chrono::high_resolution_clock::now();
    
    double result = calculate(100'000'000, 4.0, 1.0) * 4.0;
    
    auto end_time = std::chrono::high_resolution_clock::now();
    std::chrono::duration<double> execution_time = end_time - start_time;

    std::cout << std::fixed << std::setprecision(12);
    std::cout << "Result: " << result << std::endl;
    std::cout << "Execution Time: " << execution_time.count() << " seconds" << std::endl;

    return 0;
}
```

In [14]:
exec(pi)

Result: 3.141592658589
Execution Time: 15.568280 seconds


# Compiling C++ and executing

This next cell contains the command to compile a C++ file on M1 Mac.  
It compiles the file `optimized.cpp` into an executable called `optimized`  
Then it runs the program called `optimized`

In [15]:
# Compile C++ and run the executable

# !clang++ -O3 -std=c++17 -march=armv8.3-a -o optimized optimized.cpp
# !./optimized

On Windows

In [16]:
!g++ optimized.cpp -o optimized -O3 -std=c++17

!optimized

Result: 3.141592658589
Execution Time: 0.214469800000 seconds


In [17]:
# optimize_claude(pi)

In [18]:
optimize_gemini(pi)

```cpp
#include <iostream>
#include <iomanip>
#include <chrono>

using namespace std;

double calculate(int64_t iterations, int64_t param1, int64_t param2) {
    double result = 1.0;
    for (int64_t i = 1; i <= iterations; ++i) {
        double j = static_cast<double>(i * param1 - param2);
        result -= (1.0 / j);
        j = static_cast<double>(i * param1 + param2);
        result += (1.0 / j);
    }
    return result;
}

int main() {
    auto start_time = chrono::high_resolution_clock::now();
    double result = calculate(100000000, 4, 1) * 4;
    auto end_time = chrono::high_resolution_clock::now();

    auto duration = chrono::duration_cast<chrono::microseconds>(end_time - start_time);
    double seconds = duration.count() / 1000000.0;

    cout << fixed << setprecision(12) << "Result: " << result << endl;
    cout << fixed << setprecision(6) << "Execution Time: " << seconds << " seconds" << endl;

    return 0;
}
```


In [19]:
# Repeat for ClaudGemini - again, use the right approach for your platform

!g++ optimized.cpp -o optimized -O3 -std=c++17
!optimized

Result: 3.141592658589
Execution Time: 0.202344 seconds


In [20]:
python_hard = """# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value

def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""

In [21]:
exec(python_hard)

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 57.129757 seconds


In [22]:
optimize_gpt(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <chrono>
#include <limits>
#include <stdint.h>

class LCG {
public:
    LCG(uint32_t seed, uint32_t a = 1664525, uint32_t c = 1013904223, uint32_t m = 1U << 32)
        : a(a), c(c), m(m), value(seed) {}

    uint32_t next() {
        value = (a * value + c) % m;
        return value;
    }

private:
    uint32_t a, c, m, value;
};

int64_t max_subarray_sum(int n, uint32_t seed, int min_val, int max_val) {
    LCG lcg(seed);
    std::vector<int> random_numbers(n);
    
    for (int i = 0; i < n; ++i) {
        random_numbers[i] = lcg.next() % (max_val - min_val + 1) + min_val;
    }

    int64_t max_sum = std::numeric_limits<int64_t>::min();
    for (int i = 0; i < n; ++i) {
        int64_t current_sum = 0;
        for (int j = i; j < n; ++j) {
            current_sum += random_numbers[j];
            if (current_sum > max_sum) {
                max_sum = current_sum;
            }
        }
    }
    return max_sum;
}

int64_t tota

In [23]:
!g++ optimized.cpp -o optimized -O3 -std=c++17
!optimized

optimized.cpp:10:87: warning: left shift count >= width of type [-Wshift-count-overflow]
   10 |     LCG(uint32_t seed, uint32_t a = 1664525, uint32_t c = 1013904223, uint32_t m = 1U << 32)
      |                                                                                    ~~~^~~~~
optimized.cpp: In function 'int main()':
optimized.cpp:66:59: error: 'setprecision' is not a member of 'std'
   66 |     std::cout << "Execution Time: " << std::fixed << std::setprecision(6) << elapsed.count() << " seconds" << std::endl;
      |                                                           ^~~~~~~~~~~~
optimized.cpp:5:1: note: 'std::setprecision' is defined in header '<iomanip>'; this is probably fixable by adding '#include <iomanip>'
    4 | #include <chrono>
  +++ |+#include <iomanip>
    5 | #include <limits>


Result: 3.141592658589
Execution Time: 0.211275 seconds


In [24]:
optimize_gemini(python_hard)

```cpp
#include <iostream>
#include <vector>
#include <limits>
#include <chrono>
#include <iomanip>

using namespace std;

// LCG function
uint32_t lcg(uint32_t& seed, uint32_t a = 1664525, uint32_t c = 1013904223, uint32_t m = 0xFFFFFFFF) {
    seed = (a * seed + c) % m;
    return seed;
}

// Max subarray sum function
long long max_subarray_sum(int n, uint32_t seed, int min_val, int max_val) {
    vector<int> random_numbers(n);
    for (int i = 0; i < n; ++i) {
        random_numbers[i] = lcg(seed) % (max_val - min_val + 1) + min_val;
    }

    long long max_sum = numeric_limits<long long>::min();
    for (int i = 0; i < n; ++i) {
        long long current_sum = 0;
        for (int j = i; j < n; ++j) {
            current_sum += random_numbers[j];
            if (current_sum > max_sum) {
                max_sum = current_sum;
            }
        }
    }
    return max_sum;
}

// Total max subarray sum function
long long total_max_subarray_sum(int n, uint32_t initial_seed, int min_

In [25]:
!g++ optimized.cpp -o optimized -O3 -std=c++17
!optimized

Total Maximum Subarray Sum (20 runs): 10980
Execution Time: 0.528878 seconds


In [26]:
def stream_gpt(python):
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=messages_for(python), stream=True)
    reply = ""
    for chunk in stream:
        fragment = chunk.choices[0].delta.content or ""
        reply += fragment
        yield reply.replace('```cpp\n','').replace('```','')

In [27]:
# def stream_claude(python):
#     result = claude.messages.stream(
#         model=CLAUDE_MODEL,
#         max_tokens=2000,
#         system=system_message,
#         messages=[{"role": "user", "content": user_prompt_for(python)}],
#     )
#     reply = ""
#     with result as stream:
#         for text in stream.text_stream:
#             reply += text
#             yield reply.replace('```cpp\n','').replace('```','')

In [28]:
def stream_gemini(python):
    """Generates C++ code using Gemini API and yields the streaming response."""
    if 'gemini_model_instance' not in globals() or gemini_model_instance is None:
        yield "ERROR: Gemini model instance is not initialized."
        return

    prompt = user_prompt_for(python)
    reply = ""

    try:
        generation_config = genai.types.GenerationConfig(temperature=0.2)
        stream = gemini_model_instance.generate_content(
            prompt, generation_config=generation_config, stream=True
        )

        for chunk in stream:
            try:
                fragment = chunk.text
                if not reply and fragment.strip() == 'p':
                    continue
                reply += fragment
                temp_cleaned = reply
                if temp_cleaned.strip().startswith("```cpp"):
                   temp_cleaned = temp_cleaned[temp_cleaned.find("```cpp")+5:]
                if temp_cleaned.strip().endswith("```"):
                   temp_cleaned = temp_cleaned[:temp_cleaned.rfind("```")]
                yield temp_cleaned.strip()

            except ValueError:
                pass
            except Exception as e_chunk:
                 yield f"\nError: {e_chunk}"
                 print(f"\nError: {e_chunk}")

    except genai.types.generation_types.BlockedPromptException as bpe:
         yield f"Error: Prompt blocked. {bpe}"
         print(f"Error: Prompt blocked. {bpe}")
    except genai.types.generation_types.StopCandidateException as sce:
         yield f"Error: Generation Stop. {sce}"
         print(f"Error: Generation Stop. {sce}")
    except Exception as e:
        yield f"Error Gemini API: {e}"
        import traceback
        traceback.print_exc()

In [29]:
def optimize(python, model):
    if model == "GPT":
        stream_func = stream_gpt
    elif model == "Gemini":
        stream_func = stream_gemini
    # elif model=="Claude":
    #     stream_func = stream_claude
    else:
        raise ValueError("Unknown model")

    final_reply = ""
    try:
        for stream_so_far in stream_func(python):
            final_reply = stream_so_far
        if final_reply.startswith("Error:"):
             return final_reply

        cleaned_reply = final_reply.strip()

        if cleaned_reply.startswith("```cpp"):
            if cleaned_reply.split('\n')[0].strip() == '```cpp':
                 cleaned_reply = '\n'.join(cleaned_reply.split('\n')[1:])
            else:
                 cleaned_reply = cleaned_reply[cleaned_reply.find("```cpp") + 5:]

        if cleaned_reply.endswith("```"):
            cleaned_reply = cleaned_reply[:cleaned_reply.rfind("```")]

        cleaned_reply = cleaned_reply.strip()

        if model == "Gemini" and cleaned_reply.startswith('p') and '\n' in cleaned_reply:
            potential_code_start = cleaned_reply.split('\n', 1)[1].strip()
            if potential_code_start.startswith('#include') or potential_code_start.startswith('using namespace'):
                cleaned_reply = potential_code_start

        return cleaned_reply

    except Exception as e:
        import traceback
        error_msg = f"Unexpected error during optimisation:\n{traceback.format_exc()}"
        print(error_msg)
        return error_msg

In [30]:
with gr.Blocks() as ui:
    with gr.Row():
        python = gr.Textbox(label="Python code:", lines=10, value=python_hard)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Gemini"], label="Select model", value="GPT")
        convert = gr.Button("Convert code")

    convert.click(optimize, inputs=[python, model], outputs=[cpp])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [31]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [32]:
def execute_cpp(code):
        write_output(code)
        try:
            #Mac
            # compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "optimized", "optimized.cpp"]
            # Windows
            compile_cmd = ["g++", "optimized.cpp", "-o", "optimized", "-O3", "-std=c++17"]
            compile_result = subprocess.run(compile_cmd, check=True, text=True, capture_output=True)
            run_cmd = ["./optimized"]
            run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
            return run_result.stdout
        except subprocess.CalledProcessError as e:
            return f"An error occurred:\n{e.stderr}"

In [33]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [34]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["GPT", "Gemini"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


## Hugging Face Models

In [35]:
from huggingface_hub import InferenceClient
from transformers import AutoTokenizer

os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-hf-token')

codeqwen_client = InferenceClient("Qwen/CodeQwen1.5-7B-Chat")
codeqwen_tokenizer = AutoTokenizer.from_pretrained("Qwen/CodeQwen1.5-7B-Chat")

codegemma_client = InferenceClient("google/codegemma-7b-it")
codegemma_tokenizer = AutoTokenizer.from_pretrained("google/codegemma-7b-it")


In [36]:
def optimize_codeqwen(python_code):
    messages = messages_for(python_code)
    prompt = codeqwen_tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    response = codeqwen_client.text_generation(
        prompt,
        temperature=0.2,
        stream=True
    )

    reply = ""
    for chunk in response:
        reply += chunk
        print(chunk, end='', flush=True)

    write_output(reply)
    return reply

In [37]:
def optimize_codegemma(python_code):
    messages = messages_for(python_code)
    prompt = codegemma_tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    response = codegemma_client.text_generation(
        prompt,
        temperature=0.2,
        stream=True
    )

    reply = ""
    for chunk in response:
        reply += chunk
        print(chunk, end='', flush=True)

    write_output(reply)
    return reply

In [38]:
def stream_codeqwen(python):
    try:
        messages = messages_for(python)
        prompt = codeqwen_tokenizer.apply_chat_template(messages, tokenize=False)

        stream = codeqwen_client.text_generation(
            prompt,
            temperature=0.3,
            stream=True
        )

        reply = ""
        for chunk in stream:
            text_chunk = chunk.generated_text
            reply += text_chunk
            yield reply.replace("``````", "").strip()

    except Exception as e:
        yield f"Error: {str(e)}"



In [39]:
def messages_for_gemma(python):
    full_prompt = f"{system_message}\n\n{user_prompt_for(python)}"

    return [
        {"role": "user", "content": full_prompt}
    ]

def stream_codegemma(python):
    try:
        python_code = python.value if hasattr(python, 'value') else python

        messages = messages_for_gemma(python)
        prompt = codegemma_tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        stream = codegemma_client.text_generation(
            prompt,
            temperature=0.3,
            stream=True
        )

        reply = ""
        for chunk in stream:
            text_chunk = chunk.generated_text
            reply += text_chunk
            yield reply.replace("``````", "").strip()

    except Exception as e:
        yield f"Error: {str(e)}"


In [40]:
from huggingface_hub import model_info

print(model_info("google/codegemma-7b-it", token=os.environ["HF_TOKEN"]))

print(model_info("Qwen/CodeQwen1.5-7B-Chat", token=os.environ["HF_TOKEN"]))


ModelInfo(id='google/codegemma-7b-it', author='google', sha='078cdc51070553d1636d645c9a238f3b0914459a', created_at=datetime.datetime(2024, 3, 21, 16, 57, 43, tzinfo=datetime.timezone.utc), last_modified=datetime.datetime(2024, 8, 7, 18, 27, 26, tzinfo=datetime.timezone.utc), private=False, disabled=False, downloads=3692, downloads_all_time=None, gated='manual', gguf=None, inference=None, inference_provider_mapping=None, likes=217, library_name='transformers', tags=['transformers', 'safetensors', 'gemma', 'text-generation', 'conversational', 'license:gemma', 'autotrain_compatible', 'text-generation-inference', 'endpoints_compatible', 'region:us'], pipeline_tag='text-generation', mask_token=None, card_data={'base_model': None, 'datasets': None, 'eval_results': None, 'language': None, 'library_name': 'transformers', 'license': 'gemma', 'license_name': None, 'license_link': 'https://ai.google.dev/gemma/terms', 'metrics': None, 'model_name': None, 'pipeline_tag': 'text-generation', 'tags': 

In [41]:
def optimize(python, model):
    if model == "GPT":
        stream_func = stream_gpt
    elif model == "Gemini":
        stream_func = stream_gemini
    elif model == "CodeQwen1.5":
        stream_func = stream_codeqwen
    elif model == "CodeGemma":
        stream_func = stream_codegemma
    # elif model=="Claude":
    #     stream_func = stream_claude
    else:
        raise ValueError("Unknown model")

    final_reply = ""
    try:
        for stream_so_far in stream_func(python):
            final_reply = stream_so_far
        if final_reply.startswith("Error:"):
             return final_reply

        cleaned_reply = final_reply.strip()

        if cleaned_reply.startswith("```cpp"):
            if cleaned_reply.split('\n')[0].strip() == '```cpp':
                 cleaned_reply = '\n'.join(cleaned_reply.split('\n')[1:])
            else:
                 cleaned_reply = cleaned_reply[cleaned_reply.find("```cpp") + 5:]

        if cleaned_reply.endswith("```"):
            cleaned_reply = cleaned_reply[:cleaned_reply.rfind("```")]

        cleaned_reply = cleaned_reply.strip()

        if model == "Gemini" and cleaned_reply.startswith('p') and '\n' in cleaned_reply:
            potential_code_start = cleaned_reply.split('\n', 1)[1].strip()
            if potential_code_start.startswith('#include') or potential_code_start.startswith('using namespace'):
                cleaned_reply = potential_code_start

        return cleaned_reply

    except Exception as e:
        import traceback
        error_msg = f"Unexpected error during optimisation:\n{traceback.format_exc()}"
        print(error_msg)
        return error_msg

In [42]:
def select_sample_program(sample_program):
    if sample_program=="pi":
        return pi
    elif sample_program=="python_hard":
        return python_hard
    else:
        return "Type your Python program here"

In [43]:
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_hard, lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        with gr.Row():
            sample_program = gr.Radio(["pi", "python_hard"], label="Sample program", value="pi")
            model = gr.Dropdown(["GPT", "Gemini", "CodeQwen1.5", "CodeGemma"], label="Select model", value="GPT")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    sample_program.change(select_sample_program, inputs=[sample_program], outputs=[python])
    convert.click(optimize, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
